In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import seaborn as sb

In [2]:
file_to_open = ("election_groups.csv")
ml_df = pd.read_csv(file_to_open)
ml_df.head()

,Voter_ID,City,Zip_Code,Party,Birth_Year,Primary_Election_2020,General_Election_2020,Primary_Election_2016,General_Election_2016,Primary_Election_2012,General_Election_2012,Primary_Election_2008,General_Election_2008,Generational_Group
0,704844,SEVEN HILLS,44131,NOPTY,1959,0,t,D,t,0,t,D,t,Boomers
1,1658758,PARMA,44130,REP,1968,0,t,R,t,R,t,D,t,Generation X
2,2005536,BROADVIEW HTS,44147,REP,1976,0,t,R,t,R,t,D,t,Generation X
3,2465762,SOLON,44139,NOPTY,1991,0,f,0,f,0,t,0,f,Millennials
4,710633,INDEPENDENCE,44131,REP,1968,0,t,R,t,R,t,0,t,Generation X


In [3]:
party={"D": 0, "R": 1, "L": 2, "0": 3, "N": 3, " ": 3, "G": 4, "X": 3}
ml_df['Primary_Election_2016'] = ml_df['Primary_Election_2016'].apply(lambda x:party[x])
ml_df['Primary_Election_2012'] = ml_df['Primary_Election_2012'].apply(lambda x:party[x])
ml_df['Primary_Election_2008'] = ml_df['Primary_Election_2008'].apply(lambda x:party[x])
ml_df['Primary_Election_2020'] = ml_df['Primary_Election_2020'].apply(lambda x:party[x])


In [4]:
# Handle Categorical "Party" columns
P_ID = {"DEM": 0, "REP": 1,"LIB": 2, "NOPTY": 3, "GRE": 4}
ml_df["Party"] = ml_df["Party"].apply(lambda x:P_ID[x])
ml_df.head()

,Voter_ID,City,Zip_Code,Party,Birth_Year,Primary_Election_2020,General_Election_2020,Primary_Election_2016,General_Election_2016,Primary_Election_2012,General_Election_2012,Primary_Election_2008,General_Election_2008,Generational_Group
0,704844,SEVEN HILLS,44131,3,1959,3,t,0,t,3,t,0,t,Boomers
1,1658758,PARMA,44130,1,1968,3,t,1,t,1,t,0,t,Generation X
2,2005536,BROADVIEW HTS,44147,1,1976,3,t,1,t,1,t,0,t,Generation X
3,2465762,SOLON,44139,3,1991,3,f,3,f,3,t,3,f,Millennials
4,710633,INDEPENDENCE,44131,1,1968,3,t,1,t,1,t,3,t,Generation X


In [19]:
ml_df['Generational_Group'].unique()

array(['Boomers', 'Generation X', 'Millennials', 'Generation Z', 'Silent'],
      dtype=object)

In [20]:
Group = {"Silent": 1,"Boomers": 2, "Generation X": 3, "Millennials": 4, "Generation Z": 5}
ml_df["Generational_Group"] = ml_df["Generational_Group"].apply(lambda x:Group[x])

In [22]:
# Code the cities to unique #s
cities = {'SEVEN HILLS': 1, 'PARMA': 2, 'BROADVIEW HTS': 3, 'SOLON': 4, 'INDEPENDENCE': 5,
       'EUCLID': 6, 'CLEVELAND HTS': 7, 'GARFIELD HTS': 8, 'CLEVELAND': 9,
       'ROCKY RIVER': 10, 'WESTLAKE': 11, 'ORANGE': 12, 'PEPPER PIKE':13, 'MAPLE HTS':14,
       'EAST CLEVELAND':15, 'BROOK PARK':16, 'NORTH ROYALTON':17, 'HIGHLAND HTS':18,
       'WARRENSVILLE HTS':19, 'FAIRVIEW PARK':20, 'SOUTH EUCLID':21, 'BEREA':22,
       'MIDDLEBURG HTS':23, 'LAKEWOOD':24, 'LYNDHURST':25, 'SHAKER HTS':25,
       'BROOKLYN':27, 'OLMSTED FALLS':28, 'UNIVERSITY HTS':29, 'GATES MILLS':30,
       'MAYFIELD HTS':31, 'PARMA HTS':32, 'BEACHWOOD':33, 'RICHMOND HTS':33,
       'BRECKSVILLE':34, 'BEDFORD':35, 'BEDFORD HTS':36, 'OAKWOOD':37,
       'NORTH OLMSTED':38, 'OLMSTED TWP':39, 'STRONGSVILLE':40, 'BROOKLYN HTS':41,
       'NEWBURGH HTS':42, 'BAY VILLAGE':43, 'MORELAND HILLS':44, 'WALTON HILLS':45,
       'CUYAHOGA HTS':46, 'VALLEY VIEW':47, 'MAYFIELD VILLAGE':48, 'BENTLEYVILLE':49,
       'GLENWILLOW':50, 'CHAGRIN FALLS':51, 'HUNTING VALLEY':52, 'BRATENAHL':53,
       'WOODMERE':54, 'HIGHLAND HILLS':55, 'NORTH RANDALL':56, 'LINNDALE': 57,
       'CHAGRIN FALLS TWP': 58}

ml_df["City"] = ml_df["City"].apply(lambda x:cities[x])
# Set voter_id_org column as index

ml_df =ml_df.set_index('Voter_ID')
ml_df.head()


,City,Zip_Code,Party,Birth_Year,Primary_Election_2020,General_Election_2020,Primary_Election_2016,General_Election_2016,Primary_Election_2012,General_Election_2012,Primary_Election_2008,General_Election_2008,Generational_Group
Voter_ID,,,,,,,,,,,,,
704844,1,44131,3,1959,3,1,0,1,3,1,0,1,2
1658758,2,44130,1,1968,3,1,1,1,1,1,0,1,3
2005536,3,44147,1,1976,3,1,1,1,1,1,0,1,3
2465762,4,44139,3,1991,3,0,3,0,3,1,3,0,4
710633,5,44131,1,1968,3,1,1,1,1,1,3,1,3


In [7]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf


In [23]:
ml_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870075 entries, 704844 to 3017222
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   City                   870075 non-null  int64
 1   Zip_Code               870075 non-null  int64
 2   Party                  870075 non-null  int64
 3   Birth_Year             870075 non-null  int64
 4   Primary_Election_2020  870075 non-null  int64
 5   General_Election_2020  870075 non-null  int64
 6   Primary_Election_2016  870075 non-null  int64
 7   General_Election_2016  870075 non-null  int64
 8   Primary_Election_2012  870075 non-null  int64
 9   General_Election_2012  870075 non-null  int64
 10  Primary_Election_2008  870075 non-null  int64
 11  General_Election_2008  870075 non-null  int64
 12  Generational_Group     870075 non-null  int64
dtypes: int64(13)
memory usage: 92.9 MB


In [24]:
# Remove 2020 General Election target from features data
y = ml_df.General_Election_2020
X = ml_df.drop(columns=["General_Election_2020"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.766


In [26]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-07-26 18:45:08.906464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
20393/20393 [==============================] - 100s 5ms/step - loss: 0.4475 - accuracy: 0.7684
Epoch 2/50
20393/20393 [==============================] - 100s 5ms/step - loss: 0.4434 - accuracy: 0.7713
Epoch 3/50
20393/20393 [==============================] - 108s 5ms/step - loss: 0.4423 - accuracy: 0.7716
Epoch 4/50
20393/20393 [==============================] - 141s 7ms/step - loss: 0.4416 - accuracy: 0.7722
Epoch 5/50
20393/20393 [==============================] - 109s 5ms/step - loss: 0.4413 - accuracy: 0.7725
Epoch 6/50
20393/20393 [==============================] - 84s 4ms/step - loss: 0.4409 - accuracy: 0.7731
Epoch 7/50
20393/20393 [==============================] - 80s 4ms/step - loss: 0.4406 - accuracy: 0.7732
Epoch 8/50
20393/20393 [==============================] - 73s 4ms/step - loss: 0.4404 - accuracy: 0.7732
Epoch 9/50
20393/20393 [==============================] - 92s 5ms/step - loss: 0.4400 - accuracy: 0.7739
Epoch 10/50
20393/20393 [=========================